In [13]:
import pandas as pd
import pandas_datareader as web
from datetime import datetime as dt
import datetime
import calendar 
import numpy as np
import eikon as ek
ek.set_app_key('f01a5002bf4649578cf705c13f7d30d9bdcf2c9d')

In [14]:
filePath = r'C:/Users/Keavan/Jupyter/Economics/fed_funds'

In [31]:
def ff_pricing(instrument):
    
    try:
        #instrument = input('Enter instrument: ')
        futures = pd.read_csv(filePath + r'/data/{}_daily.csv'.format(instrument), header=0)
        futures = futures.set_index('Date')

        ff_codes = pd.read_csv(filePath + r'/ref/ff_codes.csv')
        contract_month = int(ff_codes['month'][ff_codes['ric'] == instrument])
        today = dt.today()
        end = today.replace(day=calendar.monthrange(today.year,contract_month)[1])
        start = end.replace(day=1)

        #EFFR Calculations
        effr = web.DataReader('EFFR', 'fred', start, end)
        average_effr_to_date = effr['EFFR'].mean()

        #Fed Funds calculations
        price_data, err = ek.get_data(instrument, fields = 'CF_LAST')
        current_price = price_data['CF_LAST'][0]

        implied_yield = 100 - current_price
        difference = average_effr_to_date - implied_yield

        return 'The difference between the Fed Funds Futures arithmetic average of the yield and the yield implied by the market is {0:.2f}%'.format(difference)

    
    except IOError:
        print('Error finding file.')
    
    except Exception as e: print(e)

In [32]:
ff_pricing('FFG0')

'The difference between the Fed Funds Future arithmetic average of the yield and the yield implied by the market is 0.00%'